In [1]:
!pip install transformers
!pip install torch
!pip install accelerate
!pip install einops

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
import os
from typing import Tuple, List
import requests
import gc

import cv2
import zipfile
from PIL import Image
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
class Model:
    """
    A class representing a model for keyframe captioning.

    Attributes:
        model_id (str): The ID of the model.
        revision (str): The revision of the model.
        model (AutoModelForCausalLM): The pretrained model for caption generation.
        tokenizer (AutoTokenizer): The tokenizer for the model.
        prompt (str): The prompt for generating captions.

    Methods:
        __init__(model_id: str, revision: str): Initializes the Model object.
        init_model(model_id: str, revision: str, cache_dir: str = "./model_cache") -> Tuple[AutoModelForCausalLM, AutoTokenizer]: Initializes the model and tokenizer.
        encode_image(image) -> torch.Tensor: Encodes the input image.
        inference(enc_image: torch.Tensor, prompt: str) -> str: Performs inference to generate a caption.
    """

    def __init__(self, model_id: str, revision: str):
        """
        Initializes a Model object.

        Args:
            model_id (str): The ID of the model.
            revision (str): The revision of the model.
        """
        self.model_id = model_id
        self.revision = revision
        self.model, self.tokenizer = self.init_model(model_id, revision)
        self.prompt = "Describe the image with as much detail as possible. Generate as much information that can be turned into meta data as possible."

    def init_model(self, model_id: str, revision: str, cache_dir: str = "./model_cache") -> Tuple[AutoModelForCausalLM, AutoTokenizer]:
        """
        Initializes the model and tokenizer.

        Args:
            model_id (str): The ID of the model.
            revision (str): The revision of the model.
            cache_dir (str, optional): The directory to cache the model. Defaults to "./model_cache".

        Returns:
            model (AutoModelForCausalLM): The pretrained model for caption generation.
            tokenizer (AutoTokenizer): The tokenizer for the model.
        """
        device = "cuda" if torch.cuda.is_available() else "cpu"
        torch_type = torch.float16 if device == "cuda" else torch.float32

        model_dir = os.path.join(cache_dir, model_id.replace('/', '_'), revision)

        # Create the directory if it does not exist
        os.makedirs(model_dir, exist_ok=True)

        if not os.path.exists(os.path.join(model_dir, 'pytorch_model.bin')):
            # Model is not saved locally, download and save it
            model = AutoModelForCausalLM.from_pretrained(
                model_id, trust_remote_code=True, revision=revision,
                torch_dtype=torch_type
            )
            tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)
        else:
            # Load the model and tokenizer from the local disk
            model = AutoModelForCausalLM.from_pretrained(
                model_dir, torch_dtype=torch_type
            )
            tokenizer = AutoTokenizer.from_pretrained(model_dir)

        model = model.to(device)
        return model, tokenizer

    def encode_image(self, image) -> torch.Tensor:
        """
        Encodes the input image.

        Args:
            image: The input image.

        Returns:
            enc_image: The encoded image.
        """
        enc_image = self.model.encode_image(image)
        return enc_image

    def run_inference(self, enc_image: torch.Tensor, prompt: str) -> str:
        """
        Performs inference to generate a caption.

        Args:
            enc_image: The encoded image.
            prompt (str): The prompt for generating captions.

        Returns:
            prompt_response: The generated caption.
        """
        prompt_response = self.model.answer_question(enc_image, prompt, self.tokenizer)
        return prompt_response

In [4]:
def get_column_values(csv_file: str, column_name: str) -> List[str]:
    df = pd.read_csv(csv_file)
    return df[column_name].values.tolist()


def get_filepaths_from_csv(csv_file: str, filename_column: str, directory: str = "videos/keyframes") -> List[str]:
    csv_filepath = os.path.join(directory, csv_file)
    filenames = get_column_values(csv_filepath, filename_column)
    full_filepaths = [f"{os.path.join(directory,''.join(filename.split('_')[:-1]),filename)}" for filename in filenames]
    return full_filepaths


def caption_images(model: Model, prompt: str, csv_file: str = "extracted_keyframes.csv", filename_column: str = 'Filename', directory: str = "videos/keyframes") -> None:
    """
    Caption the images using the provided model and prompt.

    Args:
        model (object): The image captioning model.
        prompt (str): The prompt to use for generating captions.
        csv_file (str, optional): The name of the CSV file to store the captions. Defaults to "extracted_keyframes.csv".
        filename_column (str, optional): The name of the column in the CSV file that contains the filenames. Defaults to 'Filename'.
        directory (str, optional): The directory where the keyframe images are located. Defaults to "videos/keyframes".

    Returns:
        None
    """

    full_filepaths = get_filepaths_from_csv(csv_file, filename_column, directory)

    caption_list = []

    for filepath in full_filepaths:
        image = Image.open(filepath)
        enc_image = model.encode_image(image)
        prompt_response = model.run_inference(enc_image, prompt)
        caption_list.append(prompt_response)

    csv_filepath = os.path.join(directory, csv_file)
    df = pd.read_csv(csv_filepath)
    df["Caption"] = caption_list
    df.to_csv(csv_filepath, index=True)


In [5]:
def unzip_files(zip_file, output_directory):
    with zipfile.ZipFile(zip_file, 'r') as zipf:
        zipf.extractall(output_directory)

input_dir = '/content/keyframes.zip'
output_dir = '/content/keyframes'

unzip_files(input_dir, output_dir )

In [6]:
csv_filepath = "extracted_keyframes.csv"

filename_column = 'Filename'
directory = os.path.join("/content","keyframes")

model_id = "vikhyatk/moondream2"
revision = "2024-05-20"
model = Model(model_id, revision)
prompt = "Describe the image with as much detail as possible. Generate as much information that can be turned into meta data as possible"

caption_images(
    model=model,
    prompt=prompt,
    directory=directory
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/319 [00:00<?, ?B/s]

configuration_moondream.py:   0%|          | 0.00/3.41k [00:00<?, ?B/s]

moondream.py:   0%|          | 0.00/5.29k [00:00<?, ?B/s]

vision_encoder.py:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
